In [340]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sqlalchemy import create_engine

In [12]:
time = datetime.now().replace(microsecond=0, second=0, minute=0)

In [10]:
DVL_TIMESHIFT = timedelta(days=91)
DATA_HISTORY = timedelta(days=23)

In [11]:
DVL_TIMESHIFT

datetime.timedelta(days=91)

In [ ]:
(time - DVL_TIMESHIFT).strftime("%Y-%m-%d %H:%M:%S")

'2021-11-13 14:00:00'

In [68]:
def get_lag_features(data, lag, agg_field):
    full_features = np.empty((0,len(lag)), int)
    lag_max = lag.max()
    atms = data[agg_field].unique()
    for atm in atms:
        print(atm)
        subset = data[data[agg_field] == atm]
        lag_max, mask = get_lags(subset, lag)
        print(len(mask))
        features = np.full(shape=(lag_max, len(lag)), fill_value=-1)
        features = np.append(features, subset.tr_count.values[mask], axis=0)
        full_features = np.append(full_features, features, axis=0)
    return lag_max, full_features, data.tr_count.values

def get_lags(data, lag):
    n = data.shape[0]
    lag_min, lag_max = lag.min(), lag.max()
    lags_count = n - lag_max - 1
    start_lag = lag_max - lag
    end_lag = start_lag + lags_count
    mask = np.linspace(start_lag, end_lag, lags_count+1).astype(int)

    return lag_max, mask

def fix_columns(df, date_fld):
    df[date_fld] = pd.to_datetime(df[date_fld])
    df["name"] = df["name"].astype("string")
    return df


def to_localtime(df, time_col, tz_col):
    df['local_time'] = df[time_col] + pd.to_timedelta((df[tz_col] - 3), unit='h')
    return df

def get_days_features(df, time_col):
    data = pd.DataFrame(index=df.index)
    data["friday"] = df[time_col].dt.dayofweek.values == 4
    data["saturday"] = df[time_col].dt.dayofweek.values == 5
    data["sunday"] = df[time_col].dt.dayofweek.values == 6
    data["working"] = np.bitwise_and(df[time_col].dt.dayofweek.values >= 0, df[time_col].dt.dayofweek.values <= 3)
    data["pay_day"] = np.bitwise_or(df[time_col].dt.day.values == 5, df[time_col].dt.day.values == 20)
    data["after_pay_day"] = np.bitwise_or(df[time_col].dt.day.values == 6, df[time_col].dt.day.values == 21)
    return data

def calc_tr_mode(df, w):
    '''
    params:
        df - pandas.DataFrame
        w - resampling window in format '4H'
    return: pandas.DataFrame
    '''
    d = df[["name", "local_time", "tr_count"]].groupby("name").resample(rule=w, on="local_time").mean()
    d = d.reset_index()
    d = d.groupby(["name", d.local_time.dt.hour]).mean().reset_index()
    d = pd.pivot(d, index="name", columns="local_time").reset_index()
    d = df.merge(d, on="name")
    return d.iloc[:, 3:]

In [73]:
    lag = (np.array([24, 48, 72, 96, 120, 144, 168, 336, 504]) / np.round(
        pd.Timedelta('1H').total_seconds() / 3600)).astype(int)

    nfc_lag = (np.array([24, 48, 72, 96, 120, 144, 168, 336, 504]) / np.round(
        pd.Timedelta('1D').total_seconds() / 3600)).astype(int)

In [74]:
lag

array([ 24,  48,  72,  96, 120, 144, 168, 336, 504])

In [75]:
nfc_lag

array([ 1,  2,  3,  4,  5,  6,  7, 14, 21])

In [354]:
df = pd.read_csv("../../../../data/atm_transactions/transactions_all_6683.csv")

In [355]:
df = fix_columns(df, 'date')
df = df[["name", "date", "tr_count"]]

In [294]:
atm_skto = pd.read_csv("../../../../data/atm_transactions/atm_info.csv")

In [356]:
atm_skto = atm_skto[["name", "service_time", "timezone"]]
atm_skto['name'] = atm_skto['name'].astype("string")
df = df.merge(atm_skto, on="name")
df = to_localtime(df, 'date', 'timezone')
df = df[["name", "local_time", "tr_count"]]
df = df.sort_values(["name", "local_time"])
df = df.groupby("name").resample(rule='1H', on="local_time").sum()
df = df.reset_index()

In [404]:
test_df = df[df.name.isin(['378001','378002'])]

In [443]:
idx = pd.date_range('2021-01-01 00:00:00', datetime.now().replace(microsecond=0, second=0, minute=0), freq='H')

In [444]:
idx

DatetimeIndex(['2021-01-01 00:00:00', '2021-01-01 01:00:00',
               '2021-01-01 02:00:00', '2021-01-01 03:00:00',
               '2021-01-01 04:00:00', '2021-01-01 05:00:00',
               '2021-01-01 06:00:00', '2021-01-01 07:00:00',
               '2021-01-01 08:00:00', '2021-01-01 09:00:00',
               ...
               '2022-03-01 08:00:00', '2022-03-01 09:00:00',
               '2022-03-01 10:00:00', '2022-03-01 11:00:00',
               '2022-03-01 12:00:00', '2022-03-01 13:00:00',
               '2022-03-01 14:00:00', '2022-03-01 15:00:00',
               '2022-03-01 16:00:00', '2022-03-01 17:00:00'],
              dtype='datetime64[ns]', length=10194, freq='H')

In [406]:
test_df

,name,local_time,tr_count
0,378001,2021-01-02 09:00:00,11
1,378001,2021-01-02 10:00:00,0
2,378001,2021-01-02 11:00:00,2
3,378001,2021-01-02 12:00:00,7
4,378001,2021-01-02 13:00:00,13
...,...,...,...
10150,378002,2021-08-01 19:00:00,17
10151,378002,2021-08-01 20:00:00,4
10152,378002,2021-08-01 21:00:00,4
10153,378002,2021-08-01 22:00:00,26


In [407]:
test_df.set_index(['name','local_time'], inplace=True)

In [408]:
test_df

tr_count
name   local_time                   
378001 2021-01-02 09:00:00        11
       2021-01-02 10:00:00         0
       2021-01-02 11:00:00         2
       2021-01-02 12:00:00         7
       2021-01-02 13:00:00        13
...                              ...
378002 2021-08-01 19:00:00        17
       2021-08-01 20:00:00         4
       2021-08-01 21:00:00         4
       2021-08-01 22:00:00        26
       2021-08-01 23:00:00        11

[10155 rows x 1 columns]

In [439]:
mux = pd.MultiIndex.from_product([test_df.index.levels[0], idx], 
                                 names=test_df.index.names)

In [440]:
mux

MultiIndex([('378001', '2021-01-01 00:00:00'),
            ('378001', '2021-01-01 01:00:00'),
            ('378001', '2021-01-01 02:00:00'),
            ('378001', '2021-01-01 03:00:00'),
            ('378001', '2021-01-01 04:00:00'),
            ('378001', '2021-01-01 05:00:00'),
            ('378001', '2021-01-01 06:00:00'),
            ('378001', '2021-01-01 07:00:00'),
            ('378001', '2021-01-01 08:00:00'),
            ('378001', '2021-01-01 09:00:00'),
            ...
            ('378002', '2021-08-01 15:00:00'),
            ('378002', '2021-08-01 16:00:00'),
            ('378002', '2021-08-01 17:00:00'),
            ('378002', '2021-08-01 18:00:00'),
            ('378002', '2021-08-01 19:00:00'),
            ('378002', '2021-08-01 20:00:00'),
            ('378002', '2021-08-01 21:00:00'),
            ('378002', '2021-08-01 22:00:00'),
            ('378002', '2021-08-01 23:00:00'),
            ('378002', '2021-08-02 00:00:00')],
           names=['name', 'local_time'], le

In [442]:
test_df.reindex(mux, fill_value=0).reset_index()

,name,local_time,tr_count
0,378001,2021-01-01 00:00:00,0
1,378001,2021-01-01 01:00:00,0
2,378001,2021-01-01 02:00:00,0
3,378001,2021-01-01 03:00:00,0
4,378001,2021-01-01 04:00:00,0
...,...,...,...
10221,378002,2021-08-01 20:00:00,4
10222,378002,2021-08-01 21:00:00,4
10223,378002,2021-08-01 22:00:00,26
10224,378002,2021-08-01 23:00:00,11


In [380]:
test_df.reindex(pd.date_range('2021-01-02 00:00:00', '2021-08-02 00:00:00', freq='H', closed='left')).fillna(0)

ValueError: cannot reindex from a duplicate axis

In [ ]:
df['tr_count_r'] = df.iloc[::-1].groupby('name').tr_count.rolling(4).sum().reset_index(0,drop=True).iloc[::-1]
df = df.drop(columns=["tr_count"])
df = df.rename(columns={'tr_count_r': 'tr_count'})

In [296]:
df = df[df.name.isin(['378001', '378002', '378003'])]

In [297]:
data = df.groupby('name').filter(lambda x : len(x)>lags.max())

In [298]:
metadata = data[['name', 'local_time']]
extra_data = get_days_features(data, 'local_time')

In [299]:
tr_mode_feats = calc_tr_mode(data, '4H')

/home/DVL/glotovaa/projects/data_science/lib/python3.8/site-packages/pandas/core/frame.py:9190: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  return merge(
/home/DVL/glotovaa/projects/data_science/lib/python3.8/site-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [287]:
full_features = np.empty((0,len(lag)), int)
lag_max = lag.max()
atms = data["name"].unique()

In [271]:
data

,name,local_time,tr_count
0,378001,2021-01-02 09:00:00,20.0
1,378001,2021-01-02 10:00:00,22.0
2,378001,2021-01-02 11:00:00,30.0
3,378001,2021-01-02 12:00:00,36.0
4,378001,2021-01-02 13:00:00,29.0
...,...,...,...
15228,378003,2021-08-01 18:00:00,73.0
15229,378003,2021-08-01 19:00:00,63.0
15230,378003,2021-08-01 20:00:00,NaN
15231,378003,2021-08-01 21:00:00,NaN


In [300]:
df = data

In [313]:
data2 = pd.concat([df, extra_data, tr_mode_feats], axis=1)

In [314]:
data2.shape

(15233, 24)

In [318]:
tr_mode_feats

,"(tr_count, 0)","(tr_count, 4)","(tr_count, 8)","(tr_count, 12)","(tr_count, 16)","(tr_count, 20)"
0,1.373223,12.356635,43.674528,53.811321,39.413915,8.991706
1,1.373223,12.356635,43.674528,53.811321,39.413915,8.991706
2,1.373223,12.356635,43.674528,53.811321,39.413915,8.991706
3,1.373223,12.356635,43.674528,53.811321,39.413915,8.991706
4,1.373223,12.356635,43.674528,53.811321,39.413915,8.991706
...,...,...,...,...,...,...
15228,2.516588,32.649289,96.921384,108.240566,70.398585,15.143365
15229,2.516588,32.649289,96.921384,108.240566,70.398585,15.143365
15230,2.516588,32.649289,96.921384,108.240566,70.398585,15.143365
15231,2.516588,32.649289,96.921384,108.240566,70.398585,15.143365


In [322]:
str(data2.local_time[0])

'2021-01-02 09:00:00'

In [291]:
data3.shape

(15233, 21)

In [278]:
data = pd.DataFrame(data2)
data['target'] = y
print("all data - ", data.shape)
data = data.dropna()
#data = data[(data[0] != -1)]
data = pd.concat([metadata, pd.DataFrame(data)], axis=1)
data = data.dropna()
print("data after remove zeros - ", data.shape)
y = data.target
data = data.drop(columns = "target")

all data -  (15233, 25)
data after remove zeros -  (13712, 27)
